In [ ]:
### 🔧 TAREA
#Antes de entrenar la red neuronal, modifica los hiperparámetros en `config.yaml`, especialmente:
#- `hidden_layers`: cambia el número de capas o neuronas.
#- `activation`: prueba funciones como `tanh` o `elu`.
#- `epochs`: aumenta o reduce el número de épocas.
#
#Luego, vuelve a ejecutar este notebook y observa cómo cambia el rendimiento.

In [ ]:
import os
print(os.getcwd())

import sys
sys.path.append("../src")

import train
import model
import utils
import evaluate as ev
import pandas as pd
import numpy as np
import yaml

In [ ]:
# 🔧 Cargar configuración
config = train.load_config("../config.yaml")

# 📦 Cargar y preparar los datos
X, y = train.load_data("../data/credit_clean.csv")
X_train, X_test, y_train, y_test = train.preprocess(X, y, config)

In [ ]:
# 🧠 Construir el modelo
input_dim = X_train.shape[1]
nn_model = model.build_model(config, input_dim)

In [ ]:
# 🚀 Entrenar el modelo
history = train.train_model(nn_model, X_train, y_train, config)

In [ ]:
# 💾 Guardar modelo entrenado
utils.save_model(nn_model, path="../models/nn_model.keras")

In [ ]:
# 📈 Visualizar evolución del entrenamiento
utils.plot_training_history(history, model_name="Red Neuronal")

In [ ]:
### 📊 Pregunta de reflexión
# ¿Ves signos de sobreajuste en las curvas de pérdida o precisión?  
# ¿Cuál fue la época donde la validación alcanzó su mejor punto?  
# ¿Qué pasaría si entrenaras con más (o menos) datos?